Notebook showing usage of the data detective.

In [15]:
# To import detective from relative folder
import detective.core as detective
import detective.functions as functions
import pandas as pd
pd.options.plotting.backend = "plotly"

def format_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Convert states to numeric where possible and format the last_changed."""
    df["state"] = pd.to_numeric(df["state"], errors="coerce")
    df["last_updated_ts"] = pd.to_datetime(
        df["last_updated_ts"].values, errors="ignore", utc=True, unit='s'
    ).tz_localize(None)
    df = df.dropna()
    return df

Supply the db_url as described in the [recorder integration docs](https://www.home-assistant.io/integrations/recorder/).

In [16]:
db_url = r"sqlite:///./home-assistant_v2.db"

In [17]:
db = detective.HassDatabase(db_url) # To init without fetching entities fetch_entities=False

Successfully connected to database sqlite:///./home-assistant_v2.db
There are 136 entities with data


Alternatively, detective can discover your database credentials

In [18]:
# db = detective.db_from_hass_config()

Entities are listed on an attribute

In [19]:
db.entities

['binary_sensor.hoge_retour_temperatuur',
 'device_tracker.google_maps_107844393315270166717',
 'device_tracker.google_maps_112384942883283266385',
 'device_tracker.guest',
 'device_tracker.sm_a528b',
 'input_boolean.anyone_home',
 'input_boolean.guests_at_home',
 'input_boolean.martin_asleep',
 'input_select.belichtingstype',
 'input_select.vera_s_lamp_scene',
 'light.bedlamp_anne',
 'light.bedlamp_martin',
 'light.bedlamp_thijs',
 'light.eettafel',
 'light.eettafel_1',
 'light.eettafel_2',
 'light.keuken',
 'light.kinderhoek',
 'light.lichtstrip',
 'light.overloop',
 'light.staander',
 'light.studeerkamer_bureau',
 'light.thijs_plafond_lamp',
 'light.toilet',
 'light.vensterbank',
 'light.vera_kroonluchter',
 'light.vera_kroonluchter_1',
 'light.vera_kroonluchter_2',
 'light.vera_kroonluchter_3',
 'light.wc_hal',
 'media_player.beeldstromer',
 'media_player.chromecasts_group',
 'media_player.geluidstromer',
 'media_player.huiskamer',
 'media_player.woonkamer_media',
 'persistent_noti

Use `fetch_all_data_of()` to get data for a single entity

In [20]:
df = db.fetch_all_data_of(('zone.home',))


            WITH combined_states AS (
                SELECT states.state, states.last_updated_ts, states_meta.entity_id
                FROM states
                JOIN states_meta
                ON states.metadata_id = states_meta.metadata_id
            )
            SELECT *
            FROM combined_states
            WHERE 
                entity_id IN ('zone.home')
            AND
                state NOT IN ('unknown', 'unavailable')
            ORDER BY last_updated_ts DESC
        LIMIT 50000
The returned Pandas dataframe has 79 rows of data.


In [21]:
df.head()

,state,last_updated_ts,entity_id
0,1,1.697119e+09,zone.home
1,2,1.697117e+09,zone.home
2,1,1.697117e+09,zone.home
3,2,1.697116e+09,zone.home
4,1,1.697114e+09,zone.home


Use `fetch_all_sensor_data()` to fetch all your sensor data into a pandas dataframe in memory. 

**Note** that by default the number of states returned is limited but this can optionally be over-ridden as shown below. 

In [22]:
df = db.fetch_all_sensor_data(limit=100000)


            SELECT states.state, states.last_updated_ts, states_meta.entity_id
            FROM states
            JOIN states_meta
            ON states.metadata_id = states_meta.metadata_id
            WHERE
                states_meta.entity_id  LIKE '%sensor%'
            AND
                states.state NOT IN ('unknown', 'unavailable')
            ORDER BY last_updated_ts DESC
        LIMIT 100000
The returned Pandas dataframe has 100000 rows of data.


Take a minute to read about the differences between [wide-form and long-form data](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data). The Pandas dataframe we have is in long form.

In [23]:
df

,state,last_updated_ts,entity_id
0,70.83,1.697119e+09,sensor.mi_humidity_4
1,21.01,1.697119e+09,sensor.mi_temperature_4
2,14.6830025199995,1.697119e+09,sensor.energy_consumption_tarif_2_cost
3,4175.577,1.697119e+09,sensor.gas_consumption
4,6383.134,1.697119e+09,sensor.energy_production_tarif_2
...,...,...,...
99995,-1,1.696922e+09,sensor.envoy_121944126276_current_energy_consu...
99996,12461874,1.696922e+09,sensor.envoy_121944126276_lifetime_energy_prod...
99997,44694,1.696922e+09,sensor.envoy_121944126276_last_seven_days_ener...
99998,222,1.696922e+09,sensor.envoy_121944126276_today_s_energy_produ...


It is necessary to format the data before we can plot it, and detective provides a functions to assist

In [24]:
df = format_dataframe(df)

In [25]:
df.head()

,state,last_updated_ts,entity_id
0,70.830000,2023-10-12 14:02:13.298306816,sensor.mi_humidity_4
1,21.010000,2023-10-12 14:02:13.293864960,sensor.mi_temperature_4
2,14.683003,2023-10-12 14:02:08.906647040,sensor.energy_consumption_tarif_2_cost
3,4175.577000,2023-10-12 14:02:08.902273024,sensor.gas_consumption
4,6383.134000,2023-10-12 14:02:08.896011008,sensor.energy_production_tarif_2


It is straightforward to create your own features, for example to add a `day_of_week` column

In [26]:
df['day_of_week'] = df['last_updated_ts'].apply(lambda x : x.dayofweek)
df['is_temperature'] = df['entity_id'].apply(lambda x : 'temperature' in x)

C:\Users\mwkor\AppData\Local\Temp\ipykernel_14984\2192202643.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mwkor\AppData\Local\Temp\ipykernel_14984\2192202643.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



From here you can begin plotting

In [37]:
df.sort_values(['entity_id', 'last_updated_ts'], inplace=True)
index = df.entity_id.apply(lambda x: 'humid'in x)#=="sensor.mi_humidity_4"
df[index].plot(x='last_updated_ts', y='state', color='entity_id')

C:\Users\mwkor\AppData\Local\Temp\ipykernel_14984\372781952.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
index = df.entity_id.apply(lambda x: 'temp'in x)#=="sensor.mi_humidity_4"
df[index].rename(columns={'state': 'temperature'}).plot(x='last_updated_ts', y='temperature', color='entity_id')